In [46]:
!pip install --upgrade yfinance yahooquery transformers torch spacy faker accelerate --no-cache-dir
!python -m spacy download en_core_web_lg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 156.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 52.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [47]:
import random
import yfinance as yf
import spacy
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from yahooquery import Screener
from faker import Faker
import torch


# Load SpaCy for named entity recognition
nlp = spacy.load("en_core_web_lg")

fake = Faker()

# Load Llama 3 Omni FinAI model and tokenizer
fin_ai_pipeline = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")


Device set to use cpu


In [54]:
# FUnction to generate random email attributes

def generate_random_email():
    sender_name = fake.name()
    receiver_name = fake.name()
    receiver_first_name = receiver_name.split()[0]
    sender_email = f"{sender_name.split()[0].lower()}.{sender_name.split()[1].lower()}@{random.choice(['wellsfargo.com'])}"
    receiver_email = f"{receiver_name.split()[0].lower()}.{receiver_name.split()[1].lower()}@{random.choice(['wellsfargo.com', 'bloomberg.net', 'reedsmith.com', 'bondxnapi.com', 'franklinetempleton.com', 'walkerdunlop.com', 'pimco.com'])}"
    return sender_email, receiver_email, sender_name, receiver_first_name

# Generate subject
def generate_subject(name):
    subject = "Discussion on the stock - " + name
    return subject

# Function to fetch stock summary from Yahoo Finance
def fetch_yahoo_finance_summary(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    summary = info.get('longBusinessSummary', '')
    name = info.get('longName','')
    if not summary:
        return "No summary available for this stock."
    income_statement = stock.financials
    balance_sheet = stock.balance_sheet
    cash_flow = stock.cashflow
    return summary, income_statement, balance_sheet, cash_flow, name

# Function to generate random financial data
def generate_random_financial_data():

    s = Screener()
    data = s.get_screeners("most_actives", count=100)
    tickers = [stock["symbol"] for stock in data["most_actives"]["quotes"]]
    #prices = [150.25, 302.15, 2800.60, 3345.00, 895.50, 365.20]
    sedol = [f"{random.randint(1000000, 9999999)}" for _ in range(6)]
    cusip = [f"{random.randint(10000000, 99999999)}" for _ in range(6)]
    isin = [f"US{random.randint(1000000000, 9999999999)}" for _ in range(6)]

    ticker = random.choice(tickers)
    price = yf.Ticker(ticker).info["currentPrice"]
    trade_sedol = random.choice(sedol)
    trade_cusip = random.choice(cusip)
    trade_isin = random.choice(isin)

    return ticker, price, trade_sedol, trade_cusip, trade_isin

# Function to generate a dynamic question based on the summary and extracted entities
def generate_dynamic_question(summary, name):
    
    doc = nlp(summary)
    
    ticker, price, trade_sedol, trade_cusip, trade_isin = generate_random_financial_data()
    
    entities = ', '.join(list(set([ent.text for ent in doc.ents])))
    
    # Generate a question based on the entities
    messages = [
    {
        "role": "system",
        "content": "You are a financial analyst.",
    },
    {
        "role": "user",
        "content": f"""Generate a conversation between four financial analysts named Adam, Miley, John, and Jacob. 
        They are discussing the {name} company stock in this discussion.
        The company's recent financial details are in this paragraph - {summary} and their security identifiers are ticker: {ticker}, price: {price}, sedol: {trade_sedol}, cusip: {trade_cusip}, isin: {trade_isin}, use these in the conversation.
        DO NOT include any pretext or summary mandatorily.
        Ensure the conversation is professional, insightful, and natural. 
        """
    },
]
    
    return messages



In [55]:
# Generate random financial data
ticker, price, trade_sedol, trade_cusip, trade_isin = generate_random_financial_data()

# Fetch the Yahoo Finance summary for the input ticker
summary, income_statement, balance_sheet, cash_flow, name = fetch_yahoo_finance_summary(ticker)

# Generate a dynamic question based on the summary and name
question = generate_dynamic_question(summary, name)

# Generate a random sender and receiver email addresses
sender_email, receiver_email, sender_name, receiver_first_name = generate_random_email()

# Generate the email subject
subject = generate_subject(name)

# Generate the answer using Llama 3 model
answer = fin_ai_pipeline(question, do_sample=True, temperature=0.9, return_full_text=False, num_return_sequences=1)

# Append the random trade details to the resultant answer
generated_answer = answer[0]['generated_text']

# Output the generated email
print(generated_answer)

Adam: Hey, today I have an interesting conversation with my colleagues, Miley, John, and Jacob.

Miley: Hi, Adam, how are you doing today?

Adam: I'm doing great.

John: And you, Jacob?

Jacob: Yeah, how are you?

Miley: Alright, how about our conversation?

Adam: So, as we always do, let's start with today's topic - Rigetti Computing, Inc. 

John: Yes, we've recently heard about the company. Can you tell us a little more about them?

Jacob: Sure. Rigetti Computing, Inc. Was founded in 2013 by Prof. James Kelley. The company's recent financial details show that its shares have registered a net income of -$492,212,536, which was higher than the previous year's earnings. The company currently operates in the United States and the United Kingdom.

Miley: Interesting to hear about the company's recent financial details, but can you tell us a bit more about their recent product lines?

Adam: Sure. Rigetti Computing, Inc. Offers quantum computing hardware as well as software products. Their 